Machine Learning Operations (MLOps)
Athallah Anargya Mahardika (215150201111009)

In [1]:
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-10-02 06:31:12--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-10-02 06:31:13 (15.6 MB/s) - ‘iris.csv’ saved [3975/3975]



In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')

In [3]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [10]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size=0.2)
print("Data Latih : %d" % data_latih.shape[0])
print("Data Uji : %d" % data_uji.shape[0])

Data Latih : 120
Data Uji : 30


In [11]:
label_uji = data_uji.pop('variety')

In [12]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
142,5.8,2.7,5.1,1.9
74,6.4,2.9,4.3,1.3
110,6.5,3.2,5.1,2.0
18,5.7,3.8,1.7,0.3
55,5.7,2.8,4.5,1.3


Menghitung Prior

In [13]:
from collections import Counter
def hitung_prior(list_kelas):
  n_data = len(list_kelas)
  prior = Counter(list_kelas)
  for key in prior.keys():
    prior[key] = prior[key]/n_data
  return prior

In [14]:
prior = hitung_prior(data_latih['variety'])
print(prior)

Counter({'Virginica': 0.35, 'Setosa': 0.3416666666666667, 'Versicolor': 0.30833333333333335})


Menghitung Likelihood

In [15]:
def hitung_rata2_std_kelas(input_data):
  list_columns = input_data.columns[:-1]
  class_column_name = data_latih.columns[-1]
  list_class = set(input_data[class_column_name])
  rata2 = {}
  std = {}
  for column in list_columns:
    for a_class in list_class:
      rata2[(a_class, column)] = input_data.loc[input_data[class_column_name]==a_class][column].mean()
      std[(a_class, column)] = input_data.loc[input_data[class_column_name]==a_class][column].std()
  return(rata2, std)

In [16]:
import math
def hitung_likelihood_gaussian(data, rata2, std):
  hasil = (1/math.sqrt(2*math.pi*(std**2)))*math.exp((-1*((data-rata2)**2))/(2*(std**2)))
  return hasil

Proses Training

In [17]:
def training_naive_bayes_gaussian(data_latih):
  class_column_name = data_latih.columns[-1]
  prior = hitung_prior(data_latih[class_column_name])
  (rata2, std) = hitung_rata2_std_kelas(data_latih)
  list_class = set(data_latih[class_column_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior
  model['rata2'] = rata2
  model['std'] = std
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

In [18]:
model = training_naive_bayes_gaussian(data_latih)
print(model)

{'prior': Counter({'Virginica': 0.35, 'Setosa': 0.3416666666666667, 'Versicolor': 0.30833333333333335}), 'rata2': {('Versicolor', 'sepal.length'): 5.951351351351351, ('Virginica', 'sepal.length'): 6.654761904761905, ('Setosa', 'sepal.length'): 5.014634146341463, ('Versicolor', 'sepal.width'): 2.772972972972973, ('Virginica', 'sepal.width'): 2.9976190476190476, ('Setosa', 'sepal.width'): 3.4146341463414633, ('Versicolor', 'petal.length'): 4.232432432432433, ('Virginica', 'petal.length'): 5.576190476190477, ('Setosa', 'petal.length'): 1.482926829268293, ('Versicolor', 'petal.width'): 1.337837837837838, ('Virginica', 'petal.width'): 2.0380952380952375, ('Setosa', 'petal.width'): 0.25121951219512195}, 'std': {('Versicolor', 'sepal.length'): 0.5178060670976375, ('Virginica', 'sepal.length'): 0.5972303477465409, ('Setosa', 'sepal.length'): 0.3475348727895922, ('Versicolor', 'sepal.width'): 0.34127916940749775, ('Virginica', 'sepal.width'): 0.31116468484028476, ('Setosa', 'sepal.width'): 0.39

Proses Testing

In [19]:
def testing_naive_bayes_gaussian(model, data_uji):
  prior = model['prior']
  rata2 = model['rata2']
  std = model['std']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    for column in list_columns:
      posterior[a_class] = posterior[a_class]*hitung_likelihood_gaussian(data_uji[column], rata2[(a_class, column)], std[(a_class, column)])
      posterior[a_class] = posterior[a_class]*prior[a_class]
  kelas_uji = max(posterior, key=posterior.get)
  return kelas_uji

In [20]:
indeks_uji = 0
prediksi = testing_naive_bayes_gaussian(model, data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Virginica
Virginica


In [21]:
prediksi_total = []
for indeks in range(data_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_gaussian(model, data_uji.iloc[indeks]))
print("Total prediksi benar: ", sum(prediksi_total == label_uji))

Total prediksi benar:  27
